In [1]:
using ITensors

┌ Info: Precompiling ITensors [9136182c-28ba-11e9-034c-db9fb085ebd5]
└ @ Base loading.jl:1662


In [2]:
"""
Synthesize two MPO as M2 * M1.
When using the synthesized MPO, M1 will be applied to a MPS first, then M2.
"""
function synthesize(M1::MPO, M2::MPO)
    M21 = contract(prime(M2), M1)
    prime(M21, -1, plev=2)
end

synthesize

In [3]:
# Use two qubits
nqubit = 2

# Create site indices
sites = siteinds("Qubit", nqubit)

2-element Vector{Index{Int64}}:
 (dim=2|id=360|"Qubit,Site,n=1")
 (dim=2|id=894|"Qubit,Site,n=2")

In [4]:
# Hadamard gate at qubit1, identity gate at qubit 2
# (dim=2|id=666|"Qubit,Site,n=1")が状態に作用して、(dim=2|id=666|"Qubit,Site,n=1")'が出力
layer1 = MPO(sites, ["H", "I"])
@show layer1[1]
@show layer1[2]
;

layer1[1] = ITensor ord=3
Dim 1: (dim=2|id=360|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=360|"Qubit,Site,n=1")
Dim 3: (dim=1|id=896|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×1
[:, :, 1] =
 -0.7071067811865476  -0.7071067811865475
 -0.7071067811865475   0.7071067811865475
layer1[2] = ITensor ord=3
Dim 1: (dim=2|id=894|"Qubit,Site,n=2")'
Dim 2: (dim=2|id=894|"Qubit,Site,n=2")
Dim 3: (dim=1|id=896|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×1
[:, :, 1] =
 -1.0000000000000002   0.0
  0.0                 -1.0000000000000002


In [5]:
layer2 = MPO(sites, ["X", "I"])
@show layer2[1]
@show layer2[2]
;

layer2[1] = ITensor ord=3
Dim 1: (dim=2|id=360|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=360|"Qubit,Site,n=1")
Dim 3: (dim=1|id=688|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×1
[:, :, 1] =
 0.0                 0.9999999999999998
 0.9999999999999998  0.0
layer2[2] = ITensor ord=3
Dim 1: (dim=2|id=894|"Qubit,Site,n=2")'
Dim 2: (dim=2|id=894|"Qubit,Site,n=2")
Dim 3: (dim=1|id=688|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×1
[:, :, 1] =
 1.0  0.0
 0.0  1.0


In [6]:
layer12 = synthesize(layer1, layer2)
@show layer12[1]
@show layer12[2]

layer12[1] = ITensor ord=3
Dim 1: (dim=2|id=360|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=360|"Qubit,Site,n=1")
Dim 3: (dim=1|id=617|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×1
[:, :, 1] =
 -0.9999999999999993   0.9999999999999998
 -0.9999999999999998  -0.9999999999999998
layer12[2] = ITensor ord=3
Dim 1: (dim=2|id=894|"Qubit,Site,n=2")'
Dim 2: (dim=2|id=894|"Qubit,Site,n=2")
Dim 3: (dim=1|id=617|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×1
[:, :, 1] =
 -0.7071067811865475   0.0
  0.0                 -0.7071067811865475


ITensor ord=3 (dim=2|id=894|"Qubit,Site,n=2")' (dim=2|id=894|"Qubit,Site,n=2") (dim=1|id=617|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [7]:
# Create a random MPS with bond dimension 4
M = randomMPS(sites; linkdims=4)

MPS
[1] ((dim=2|id=360|"Qubit,Site,n=1"), (dim=2|id=325|"Link,l=1"))
[2] ((dim=2|id=325|"Link,l=1"), (dim=2|id=894|"Qubit,Site,n=2"))


In [8]:
# Apply layer1 and layer2 to M in this order
res = apply(layer2, apply(layer1, M; cutoff=1e-20); cutoff=1e-20)

# To Julia Array: (qubit1, qubit2)
res_arr = Array(reduce(*, res), sites)

2×2 Matrix{Float64}:
 -0.811272  -0.366202
  0.41275   -0.193318

In [9]:
# Apply layer1 and layer2 to M in this order
res2 = apply(layer12, M; cutoff=1e-20) 

# To Julia Array: (qubit1, qubit2)
res2_arr = Array(reduce(*, res2), sites)

2×2 Matrix{Float64}:
 -0.811272  -0.366202
  0.41275   -0.193318

In [10]:
# Create Z_1 X_2
# See: https://github.com/ITensor/ITensors.jl/blob/main/src/mps/mpo.jl#L69
os = Prod{Op}()
os *= Op("Z", 1)
os *= Op("X", 2)
MPO(os, sites)

MPO
[1] ((dim=3|id=184|"Link,l=1"), (dim=2|id=360|"Qubit,Site,n=1")', (dim=2|id=360|"Qubit,Site,n=1"))
[2] ((dim=3|id=184|"Link,l=1"), (dim=2|id=894|"Qubit,Site,n=2")', (dim=2|id=894|"Qubit,Site,n=2"))


In [11]:
# Create Z_1
MPO(Op("Z", 1), sites)

MPO
[1] ((dim=3|id=224|"Link,l=1"), (dim=2|id=360|"Qubit,Site,n=1")', (dim=2|id=360|"Qubit,Site,n=1"))
[2] ((dim=3|id=224|"Link,l=1"), (dim=2|id=894|"Qubit,Site,n=2")', (dim=2|id=894|"Qubit,Site,n=2"))


In [12]:
# Control: 1
# Target: 2
cnot = op("CNOT", sites, 1, 2)
MPO(cnot, sites)

MPO
[1] ((dim=2|id=360|"Qubit,Site,n=1")', (dim=2|id=360|"Qubit,Site,n=1"), (dim=4|id=398|"Link,n=1"))
[2] ((dim=4|id=398|"Link,n=1"), (dim=2|id=894|"Qubit,Site,n=2")', (dim=2|id=894|"Qubit,Site,n=2"))


In [13]:
mps = MPS(sites, ["1", "0"])
@show mps[1]
@show mps[2]

mps[1] = ITensor ord=2
Dim 1: (dim=2|id=360|"Qubit,Site,n=1")
Dim 2: (dim=1|id=897|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×1
 0.0
 1.0
mps[2] = ITensor ord=2
Dim 1: (dim=1|id=897|"Link,l=1")
Dim 2: (dim=2|id=894|"Qubit,Site,n=2")
NDTensors.Dense{Float64, Vector{Float64}}
 1×2
 1.0  0.0


ITensor ord=2 (dim=1|id=897|"Link,l=1") (dim=2|id=894|"Qubit,Site,n=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [14]:
MPS(sites, "1")

MPS
[1] ((dim=2|id=360|"Qubit,Site,n=1"), (dim=1|id=460|"Link,l=1"))
[2] ((dim=1|id=460|"Link,l=1"), (dim=2|id=894|"Qubit,Site,n=2"))


In [40]:
sites = [Index(2, "Qubit, site=$s") for s in 1:4]

function onemps(::Type{T}, sites) where {T<:Number}
    M = MPS(T, sites; linkdims=1)
    l = linkinds(M)
    for n in eachindex(M)
        if n == 1
            M[n] = ITensor(T, sites[n], l[n])
        elseif n == length(M)
            M[n] = ITensor(T, l[n-1], sites[n])
        else
            M[n] = ITensor(T, l[n-1], sites[n], l[n])
        end
        M[n] .= one(T)
    end
    return M
end

M = onemps(Float64, sites)
M[1]
linkinds(M)

3-element Vector{Index{Int64}}:
 (dim=1|id=877|"Link,l=1")
 (dim=1|id=514|"Link,l=2")
 (dim=1|id=359|"Link,l=3")

In [42]:
Array(reduce(*, M), sites)

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 1.0  1.0
 1.0  1.0

[:, :, 2, 1] =
 1.0  1.0
 1.0  1.0

[:, :, 1, 2] =
 1.0  1.0
 1.0  1.0

[:, :, 2, 2] =
 1.0  1.0
 1.0  1.0